In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D, Dense
import pickle

# Load the dataset (adjust the path as necessary)
file_path = 'combined_data.xlsx'
data = pd.read_excel(file_path)

# Replace -1 with 0 for positive sentiment and 1 remains for negative sentiment
data.replace({-1: 0, 1: 1}, inplace=True)

# Determine the overall sentiment
data['overall_sentiment'] = data.drop(columns=['sentence']).apply(lambda row: 1 if row.sum() > 0 else 0, axis=1)

# Initialize tokenizer and fit on sentences
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['sentence'])

# Convert sentences to sequences and pad them to ensure uniform length
sequences = tokenizer.texts_to_sequences(data['sentence'])
X = pad_sequences(sequences)
y = data['overall_sentiment'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build the model
vocab_size = len(tokenizer.word_index) + 1  # Adding 1 because of reserved 0 index
max_length = X.shape[1]  # Use the padded sequence length

model = Sequential([
    Embedding(vocab_size, 16, input_length=max_length),
    GlobalAveragePooling1D(),
    Dense(24, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
236/236 [==============================] - 2s 6ms/step - loss: 0.4546 - accuracy: 0.8757 - val_loss: 0.3292 - val_accuracy: 0.8964
Epoch 2/20
236/236 [==============================] - 1s 5ms/step - loss: 0.3428 - accuracy: 0.8889 - val_loss: 0.3244 - val_accuracy: 0.8964
Epoch 3/20
236/236 [==============================] - 1s 6ms/step - loss: 0.3384 - accuracy: 0.8889 - val_loss: 0.3213 - val_accuracy: 0.8964
Epoch 4/20
236/236 [==============================] - 1s 6ms/step - loss: 0.3339 - accuracy: 0.8889 - val_loss: 0.3159 - val_accuracy: 0.8964
Epoch 5/20
236/236 [==============================] - 2s 9ms/step - loss: 0.3278 - accuracy: 0.8893 - val_loss: 0.3087 - val_accuracy: 0.8959
Epoch 6/20
236/236 [==============================] - 2s 7ms/step - loss: 0.3173 - accuracy: 0.8902 - val_loss: 0.2950 - val_accuracy: 0.8975
Epoch 7/20
236/236 [==============================] - 1s 6ms/step - loss: 0.2952 - accuracy: 0.8923 - val_loss: 0.2724 - val_accuracy: 0.9012
Epoch 

#**Calculate Accuracy**

In [25]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test Accuracy:', test_acc)

59/59 [==============================] - 0s 2ms/step - loss: 0.1499 - accuracy: 0.9485
Test Accuracy: 0.9484864473342896


In [26]:
def predict_sentiment(text):
    # Convert the sentence to a sequence
    sequence = tokenizer.texts_to_sequences([text])

    # Pad the sequence
    padded_sequence = pad_sequences(sequence, maxlen=X.shape[1])

    # Predict sentiment
    prediction = model.predict(padded_sequence)[0][0]

    # Interpret the prediction
    if prediction < 0.5:
        return "Positive sentiment (0)", prediction
    else:
        return "Negative sentiment (1)", prediction

# Example usage
sentence = "We cannot and will not be liable for any loss or damage arising from your failure to comply with the above."
sentiment, confidence = predict_sentiment(sentence)
print(f"Predicted Sentiment: {sentiment}")

sentence = "We may suspend or terminate your account or cease providing you with all or part of the Services"
sentiment, confidence = predict_sentiment(sentence)
print(f"Predicted Sentiment: {sentiment}")

sentence = "You are responsible for your use of the Services and for any Content you provide, including compliance with applicable laws, rules, and regulations."
sentiment, confidence = predict_sentiment(sentence)
print(f"Predicted Sentiment: {sentiment}")


1/1 [==============================] - 0s 69ms/step
Predicted Sentiment: Negative sentiment (1)
1/1 [==============================] - 0s 25ms/step
Predicted Sentiment: Negative sentiment (1)
1/1 [==============================] - 0s 23ms/step
Predicted Sentiment: Positive sentiment (0)


In [27]:
import nltk
from keras.preprocessing.sequence import pad_sequences
nltk.download('punkt')

def print_negative_sentences(paragraph):
    count = 0
    # Split the paragraph into sentences
    sentences = nltk.tokenize.sent_tokenize(paragraph)

    for sentence in sentences:
        # Convert the sentence to a sequence
        sequence = tokenizer.texts_to_sequences([sentence])

        # Pad the sequence
        padded_sequence = pad_sequences(sequence, maxlen=X.shape[1])

        # Predict sentiment
        prediction = model.predict(padded_sequence)[0][0]

        # If the prediction indicates a negative sentiment, print the sentence
        if prediction >= 0.5:  # Assuming 0.5 as the threshold for negative sentiment
            count = count + 1
            print(sentence)
            print(f"count : {count}")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
# Example paragraph
paragraph = "We may suspend or terminate your account or cease providing you with all or part of the Services at any time if we reasonably believe: (i) you have violated these Terms or our Rules and Policies, (ii) you create risk or possible legal exposure for us; (iii) your account should be removed due to unlawful conduct; (iv) your account should be removed due to prolonged inactivity; or (v) our provision of the Services to you is no longer commercially viable. We will make reasonable efforts to notify you by the email address associated with your account or the next time you attempt to access your account, depending on the circumstances. To the extent permitted by law, we may also terminate your account or cease providing you with all or part of the Services for any other reason or no reason at our convenience. In all such cases, the Terms shall terminate, including, without limitation, your license to use the Services, except that the following sections shall continue to apply: 2, 3, 5, 6, and the misuse provisions of Section 4 (“Misuse of the Services”). If you believe your account was terminated in error you can file an appeal following the steps found in our Help Center (https://help.x.com/forms/account-access/appeals). For the avoidance of doubt, these Terms survive the deactivation or termination of your account."

# Call the function with the example paragraph
print_negative_sentences(paragraph)

1/1 [==============================] - 0s 23ms/step
We may suspend or terminate your account or cease providing you with all or part of the Services at any time if we reasonably believe: (i) you have violated these Terms or our Rules and Policies, (ii) you create risk or possible legal exposure for us; (iii) your account should be removed due to unlawful conduct; (iv) your account should be removed due to prolonged inactivity; or (v) our provision of the Services to you is no longer commercially viable.
count : 1
1/1 [==============================] - 0s 21ms/step
To the extent permitted by law, we may also terminate your account or cease providing you with all or part of the Services for any other reason or no reason at our convenience.
count : 2
1/1 [==============================] - 0s 22ms/step


#**SAVING THE MODEL**

In [29]:
# Save model architecture as JSON
model_json = model.to_json()
with open('model_architecture.json', 'w') as json_file:
    json_file.write(model_json)

# Save model weights
model.save_weights('my_model_weights.h5')


In [30]:
from tensorflow.keras.models import load_model
import pickle

# Assuming `model` is your trained Keras model and `tokenizer` is your Keras tokenizer

# Save the model
model.save('sentiment_model.h5')

# Save the tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [31]:
file_path = 'Spotify.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

print_negative_sentences(text)

1/1 [==============================] - 0s 35ms/step
you acknowledge that you have read and understood the agreements , accept these agreements , and agree to be bound by them .
count : 1
1/1 [==============================] - 0s 54ms/step
occasionally we may , in our discretion , make changes to the agreements .
count : 2
1/1 [==============================] - 0s 56ms/step
in some cases , we will notify you in advance , and your continued use of the service after the changes have been made will constitute your acceptance of the changes .
count : 3
1/1 [==============================] - 0s 22ms/step
spotify reserves the right , in its absolute discretion , to determine your eligibility for a trial , and , subject to applicable laws , to withdraw or to modify a trial at any time without prior notice and with no liability , to the greatest extent permitted under the law .
count : 4
1/1 [==============================] - 0s 37ms/step
in all cases , spotify reserves the right to remove or d

In [32]:
file_path = 'amazon.txt'
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

print_negative_sentences(text)

1/1 [==============================] - 0s 20ms/step


#**RUNNING THE FLASK WEB SERVER**

In [33]:
# Install required packages
!pip install flask

# Import necessary libraries
from flask import Flask, request, render_template_string
import pickle
from tensorflow.keras.models import load_model
import nltk
from keras.preprocessing.sequence import pad_sequences

# Download the required NLTK data
nltk.download('punkt')

# Load the saved model and tokenizer
model = load_model('sentiment_model.h5')
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

# Create Flask app
app = Flask(__name__)

# Define the route for the main page
@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST':
        text = request.form['text']
        negative_sentences, count = analyze_sentiment(text)
        return render_template_string(html_template, negative_sentences=negative_sentences, count=count)
    return render_template_string(html_template)

# Function to analyze sentiment
def analyze_sentiment(paragraph):
    negative_sentences = []
    count = 0
    sentences = nltk.tokenize.sent_tokenize(paragraph)
    for sentence in sentences:
        sequence = tokenizer.texts_to_sequences([sentence])
        padded_sequence = pad_sequences(sequence, maxlen=X.shape[1])
        prediction = model.predict(padded_sequence)[0][0]
        if prediction >= 0.5:
            count += 1
            negative_sentences.append(f'<span style="color:red;">{sentence}</span>')
    return negative_sentences, count

# HTML template
html_template = """
<!DOCTYPE html>
<html>
  <head>
    <title>Sentiment Analysis</title>
    <style>
      body {
        font-family: Arial, sans-serif;
        background-color: #f5f5f5;
        padding: 20px;
      }
      h1 {
        color: #333;
        text-align: center;
      }
      form {
        max-width: 600px;
        margin: 0 auto;
        background-color: #fff;
        padding: 20px;
        border-radius: 5px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
      }
      textarea {
        width: 100%;
        height: 150px;
        padding: 10px;
        font-size: 16px;
        border: 1px solid #ccc;
        border-radius: 4px;
        box-sizing: border-box;
      }
      input[type=submit] {
        background-color: #4CAF50;
        color: white;
        padding: 10px 20px;
        border: none;
        border-radius: 4px;
        cursor: pointer;
        font-size: 16px;
      }
      input[type=submit]:hover {
        background-color: #45a049;
      }
      .result {
        max-width: 600px;
        margin: 20px auto 0;
        background-color: #fff;
        padding: 20px;
        border-radius: 5px;
        box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
      }
    </style>
  </head>
  <body>
    <h1>Sentiment Analysis</h1>
    <form method="post">
      <label for="text">Enter text:</label><br>
      <textarea name="text" rows="5" cols="50"></textarea><br>
      <input type="submit" value="Analyze">
    </form>
    {% if negative_sentences %}
    <div class="result">
      <p><strong>Total number of negative sentences: {{ count }}</strong></p>
      <p>Negative sentences:</p>
      {% for sentence in negative_sentences %}
      <p>{{ sentence|safe }}</p>
      {% endfor %}
    </div>
    {% endif %}
  </body>
</html>
"""

# Run the Flask app
if __name__ == '__main__':

    from google.colab.output import eval_js
    print(eval_js("google.colab.kernel.proxyPort(5000)"))
    app.run()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


https://8lupf3wkc9n-496ff2e9c6d22116-5000-colab.googleusercontent.com/
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 06:38:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 06:38:49] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 22ms/step


INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 06:39:26] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 06:39:26] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 0s 21ms/step


INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 06:41:57] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [02/Apr/2024 06:41:58] "GET /favicon.ico HTTP/1.1" 404 -
